# Face detection with OpenCV using Python  

#### Ziele des Lückenkernels:
* Grobe Einführung in OpenCV
* Face Detection
    * Grobe Einführung in Haar-Cascades
    
### Face detection
* Face detection ist eine Aufgabe der Bildanalyse
* Das Ziel ist eine Lokalisierung eines gesuchten Objektes -> in diesem Fall ein Gesicht
* Die Aufgabe unterteilt sich dabei in 2 Sub-Aufgaben
    * 1. Klassifikation des Objektes
    * 2. Lokalisierung durch eine Bounding Box oder ähnliches

#### Durchführung des Jupyter Notebooks
* Die Zellen, die mit **#Vorbereitung** beginnen können Sie einfach laufen lassen. Dazu klicken Sie einfach die jeweilige Zelle an und klicken anschließend auf Run (befindet sich im Reiter).
* Für die Bearbeitung des Notebooks müssen Sie jede Zelle **nacheinander** durchgehen.
* Dabei gibt es Zellen, die beinhalten **nur Informationen, Code den Sie nur ausführen müssen (Zellen, die mit "#Vorbereitung" beginnen und Zellen, in denen Sie selber Code schreiben müssen.**
* **Beachten Sie bitte die Kommentare!**


#### Daten
* Im Lückenkernel-Verzeichnis (das ist bei Ihnen ein Tab/Fenster) sehen Sie den Ordner **images**.
* Darin befinden sich die Gesichter, die hier identifiziert werden.
* Schauen Sie sich doch mal die Bilder an. ;-)

#### Output
* Im Lückenkernel-Verzeichnis befindet sich der Ordner **Output**.
* Darin wird der Output nach der Gesichtserkennung abgespeichert.


### OpenCV
* OpenCV -> **Open Source Computer Vision Libary**
* Es ist ein Open Source Package für den Fachbereich der Computer Vision


* Alle OpenCV Funktionen werden mit **cv2.** aufgerufen
* In diesem Lückenkernel werden Sie die herkömmlichen Funktionen kennenlernen. 
* Für weitere Informationen verweise ich auf die Dokumentation von OpenCV: **https://docs.opencv.org/master/d6/d00/tutorial_py_root.html**


### Haar-Cascades
* Haar Cascades ist ein **Object Detection Algorithmus**, um Gesichter in Bildern und in Echtzeit-Videos zu identifizieren.
* Haar Cascades wurden erstmals von Paul Viola und Michael Jones im Jahr 2001 in einem Paper vorgestellt.
    * **"Rapid Object Detection using a Boosted Cascade of Simple Features"**
* Es gewann vor allem durch seine hohe Effizienz Muster in Echtzeit zu erkennen an Popularität.
* OpenCV stellt bereits eine Auswahl an Haar Cascades in XML-Datei zur Verfügung. Das sind sozusagen nichts anderes als Modelle mit vordefinierten Features.
* Es besteht auch die Möglichkeit eigene Haar Cascades zu erstellen, in dem man mehrere Haar Cascades miteinander kombiniert.


* Haar Cascades besitzen sogenannte **Features**. Das ist nichts anderes als **Filter** (oder auch **Kernel** genannt).
* Die untere Abbildung zeigt einige Variationen von Features - Es gibt immer einen **hellen und dunklen Teil**

<img src="Markdown_imgs/Features.png">

* Haar Cascades verwenden diese Features, um **Kanten, Linien oder ähnliches** zu erkennen. Diese Strukturen sind aber auf die Form eines Gesichts spezifiziert. D.h. dass andere Objekte, die zum Beispiel im Hintergrund sind, werden (sollten) ignoriert werden.

* Die Features wandern über jedes Fenster, dass über das Bild gelegt wird. 
* Pro Fenster wird dann ein Feature (kann auch ein ganzes Set von verschiedenen Features sein) angewendet.
* Haar Cascades können dabei **mehrere Sets von Features** haben, die wiederum in verschiedene **"Stages"** vorhanden sind. 
* **Stages** werden verwendet, um die Effizienz zu steigern und die Rechenzeit zu verrringern. 
* Fällt ein Fenster durch eine Stage durch, dann werden die nachfolgenden Stages nicht mehr angewendet. 
* D.h., dass das Fenster **keine relevante Gesichtsstruktur beinhaltet.**


Die folgende Abbildung zeigt die Berechnungsmethodik eines eingesetzten Features auf ein Fenster.

<img src="Markdown_imgs/Use_features.png">

Es wird jeweils der Mittelwert der Pixel in der hellen und der dunklen Fläche berechnet und anschließend die Differenz berechnet. **Ist diese Differenz nahe dem Wert 1, dann handelt es sich um eine relevante Struktur.** Ansonsten ist nichts relevantes dabei.

In [1]:
#Vorbereitung

# Import der Packages
import cv2 #Das ist OpenCV
import os
import shutil
import pickle

In [2]:
#Vorbereitung

# Die Haar Cascades werden in das Verzeichnis kopiert, in dem gearbeitet wird.
# Das ist ein Teil der Vorbereitung.
# Sie müssen hier nix machen


# Mit cv2.__file__ erhält man den Speicherort von cv2 -> da sind auch die Cascades gespeichert
cascade_rootpath = cv2.__file__

# cascade_datapath enthält ausschließlich die Cascades Dateien - Ziel ist es nun diese Daten in unser Arbeitsverzeichnis zu laden
cascade_datapath = cascade_rootpath[:-len(cascade_rootpath.split("/")[-1])]+"data"

# cascade_dstpath ist unser Arbeitsverzeichnis
cascade_dstpath = os.getcwd()+"/cascades"

# Die Cascades werden hier in das Arbeitsverzeichnis geladen
shutil.rmtree(cascade_dstpath)
shutil.copytree(cascade_datapath, cascade_dstpath)

'/Users/lookphanthavong/Documents/VSCode/Lückenkernel/cascades'

In [3]:
# Geben Sie nun alle Cascades aus - Achtung es ist ein Verzeichnis (Tipp: os.listdir())
# Ein print() muss nicht angegeben werden.


### BEGIN SOLUTION
os.listdir(cascade_dstpath)
### END SOLUTION



['haarcascade_upperbody.xml',
 'haarcascade_frontalcatface_extended.xml',
 'haarcascade_profileface.xml',
 'haarcascade_frontalcatface.xml',
 'haarcascade_frontalface_alt2.xml',
 'haarcascade_eye.xml',
 '__init__.py',
 'haarcascade_lefteye_2splits.xml',
 '__pycache__',
 'haarcascade_frontalface_alt_tree.xml',
 'haarcascade_licence_plate_rus_16stages.xml',
 'haarcascade_righteye_2splits.xml',
 'haarcascade_frontalface_alt.xml',
 'haarcascade_lowerbody.xml',
 'haarcascade_russian_plate_number.xml',
 'haarcascade_frontalface_default.xml',
 'haarcascade_smile.xml',
 'haarcascade_fullbody.xml',
 'haarcascade_eye_tree_eyeglasses.xml']

Sie sehen nun eine Liste an verschiedenen Haar Cascades, die OpenCV zur Verfügung stellt.

In [4]:
# Mit cv2.CascadeClassifier() können Sie die Cascade laden - Geben Sie unbedingt den absoluten Pfad an
# Um den absoluten Pfad anzugeben, verwenden Sie bitte die os.path.join(source_path, Dateiname) Funktion
# Als Beispiel verwenden Sie bitte "haarcascade_frontalface_default.xml"
# Speichern Sie den CascadeClassifier als die Variable face_cascade ab




### BEGIN SOLUTION
face_cascade = cv2.CascadeClassifier(os.path.join(cascade_dstpath,'haarcascade_frontalface_default.xml'))
### END SOLUTION




### Allgemeine Vorgehensweise
* In dem unteren Abschnitt findet die Gesichtserkennung statt.
* Zunächst wird das Bild eingelesen und in ein **Grauwertbild umgewandelt**, da die Haar Cascades nur mit Grauwertbildern arbeiten kann.
* Anschließend wird mit **face_cascade.detectMultiScale()** die Gesichter erkannt.
* face_cascade.detectMultiScale liefert dabei nur die Koordinaten einer "Box" in der sich da Gesicht befindet. Die Box enthält **4 Koordinaten x, y, w und h**. Eine genauere Beschreibung der Koordinaten wird weiter unten erklärt.
* Mit diesen Koordinaten kann eine Box um das Gesicht gezeichnet werden. Dazu wird die **cv2.rectangle()** Funktion verwendet.
* Anschließend können die Bilder mit **cv2.imwrite()** gespeichert werden.

In [7]:
# Wir legen uns ein Output-Verzeichnis an
recog_output_path = os.getcwd()+"/Output"

# Die verwendeten Bilder befinden sich im Ordner "images"
img_path = os.getcwd()+"/images"


dirs = os.listdir(img_path)
for name in dirs:
    if name == ".ipynb_checkpoints" or name == ".DS_Store":
        pass
    else:
        list_of_imgs = os.listdir(os.path.join(img_path, name))
        for img_name in list_of_imgs:
            if img_name.endswith(".jpg"):
                total_path = img_path+"/"+name+"/"+img_name
            
                
############################################################################################################
# Um die Bilder einlesen zu können, wird der absolute Pfad für jedes Bild benötigt.
# total_path stellt den absoluten Pfad des Bildes dar - es ist immer nur der Pfad von einem Bild !!!
# Laden Sie mit cv2.imread() das Bild und speichern Sie das Bild als img
# HINWEIS: Sie befinden sich in einer Schleife - Achten Sie daher auf die richtige Einrückung



                ### BEGIN SOLUTION
                img = cv2.imread(total_path)
                ### END SOLUTION



# Cascaden arbeiten ausschließlich nur mit Grauwertbildern
# Mit cv2.cvtColor() können Sie das eingelesene Bild in ein Grauwertbild umwandeln
# Hier ist ein Link der Ihnen eine Auswahl an Konvertierungen darstellt: https://www.geeksforgeeks.org/python-opencv-cv2-cvtcolor-method/
# Verwenden Sie cv2.COLOR_BGR2GRAY als Konvertierung und speichern Sie das Grauwertbild als gray ab



                ### BEGIN SOLUTION
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                ### END SOLUTION



# Nun können Sie mit der Funktion face_cascade.detectMultiScale() das Graubild einlesen und Gesichter erkennen
# Verwenden Sie diese Funktion mit den Parametern scaleFactor=1.5 und minNeighbors=5 und speichern Sie die "Gesichter" als faces ab



                ### BEGIN SOLUTION
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
                ### END SOLUTION



# faces enthält lediglich die Koordinaten der sogenannten REGION OF INTEREST ROI 
# ROI stellt wiederum das Gesicht dar 
# faces enthält daher lediglich nur die Koordination dieser ROI
# Koordinate 1: x->Startpunkt auf der X-Achse
# Koordinate 2: y-> Startpunkt auf der Y-Achse
# Koordinate 3: w-> Endpunkt auf der X-Achse; w steht für width (=Breite)
# Koordinate 4: h-> Endpunkt auf der Y-Achse; h steht für height (=Höhe)
# Gehen Sie nun mit einer for-loop durch faces mit x,y,w,h als Iteratoren (Hinweis: Klammern nicht vergessen!!!)
# Anschließend könne Sie mit cv2.rectangle() und den Koordinaten der ROI eine Box zeichnen, die auf das RGB Bild (=img) zusehen ist
# Die Parameter für cv2.rectangle sind: Bild, (Startp.X-Achse, Startp.Y-Achse), (Breite, Höhe) -> Achtung: Der Startpunkt gehört dazu!!!, (255,0,0)->b,g,r, 2 -> Als Rahmenbreite



                ### BEGIN SOLUTION
                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                ### END SOLUTION



# Mit cv2.imwrite() können Sie die Bilder mit Box gespeichert werden

                cv2.imwrite(os.path.join(recog_output_path , name+"_"+img_name), img)
        

/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Emma_Watson/4.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Emma_Watson/5.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Emma_Watson/2.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Emma_Watson/3.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Emma_Watson/1.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Daniel_Radcliffe/4.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Daniel_Radcliffe/5.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Daniel_Radcliffe/2.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Daniel_Radcliffe/3.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Daniel_Radcliffe/1.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Rupert_Grint/4.jpg
/Users/lookphanthavong/Documents/VSCode/Lückenkernel/images/Rupert_Grint/5.jpg
/Users/lookphanthavong/Do